# Лабораторна робота 2
### Виконав Терещенко Микола ФБ-33
<b> Мета роботи:</b> ознайомитися з основними кроками по роботі з даними – workflow від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі та природу даних, над якими виконується аналітичні операції

1. Для кожної із адміністративних одиниць України завантажити текстові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження;

In [3]:
import os
import pandas as pd
import urllib.request
from datetime import datetime

In [4]:
path = '/home/kali/Downloads/csv'
def get_data(province_id):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1=1981&year2=2024&type=Mean"
    vhi_url = urllib.request.urlopen(url)

    now = datetime.now().strftime('%Y-%m-%d_%H-%M')
    # path = '/home/kali/Downloads/csv'
    filename = f'vhi_id_{province_id}_{now}.csv'
    filepath = f'{path}/{filename}'
    with open(filepath, 'wb') as out:
        out.write(vhi_url.read())
    print(f"VHI was downloaded in {filepath}")

    return

for i in range(1,28):
    get_data(i)

VHI was downloaded in /home/kali/Downloads/csv/vhi_id_1_2025-03-24_09-47.csv
VHI was downloaded in /home/kali/Downloads/csv/vhi_id_2_2025-03-24_09-47.csv
VHI was downloaded in /home/kali/Downloads/csv/vhi_id_3_2025-03-24_09-47.csv
VHI was downloaded in /home/kali/Downloads/csv/vhi_id_4_2025-03-24_09-47.csv
VHI was downloaded in /home/kali/Downloads/csv/vhi_id_5_2025-03-24_09-47.csv
VHI was downloaded in /home/kali/Downloads/csv/vhi_id_6_2025-03-24_09-47.csv
VHI was downloaded in /home/kali/Downloads/csv/vhi_id_7_2025-03-24_09-47.csv
VHI was downloaded in /home/kali/Downloads/csv/vhi_id_8_2025-03-24_09-48.csv
VHI was downloaded in /home/kali/Downloads/csv/vhi_id_9_2025-03-24_09-48.csv
VHI was downloaded in /home/kali/Downloads/csv/vhi_id_10_2025-03-24_09-48.csv
VHI was downloaded in /home/kali/Downloads/csv/vhi_id_11_2025-03-24_09-48.csv
VHI was downloaded in /home/kali/Downloads/csv/vhi_id_12_2025-03-24_09-48.csv
VHI was downloaded in /home/kali/Downloads/csv/vhi_id_13_2025-03-24_09-48

In [5]:
state_id_name = {
        1: 'Vinnytsia', 13: 'Mykolaiv',
        2: 'Volyn', 14: 'Odesa',
        3: 'Dnipropetrovska', 15: 'Poltavska',
        4: 'Donetsk', 16: 'Rivne',
        5: 'Zhytomyr', 17: 'Sumy',
        6: 'Zakarpattia', 18: 'Ternopil',
        7: 'Zaporizhzhya', 19: 'Kharkiv',
        8: 'Ivano-Frankivsk', 20: 'Kherson',
        9: 'Kyiv', 21: 'Khmelnytsky',
        10: 'Kirovohrad', 22: 'Cherkasy',
        11: 'Luhansk', 23: 'Chernivtsi',
        12: 'Lviv', 24: 'Chernihiv',
        25: 'Republic of Crimea'
}

2. Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах). Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [6]:
def read_and_create_data_frame(path):
    
    out_df = pd.DataFrame()

    csv_files = os.listdir('/home/kali/Downloads/csv')

    frames = []

    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']


    for file in range(len(csv_files)):
        file_path = os.path.join(path, csv_files[file])
        df = pd.read_csv(file_path , header = 1, names = headers)
        
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df['Year'] = df['Year'].str.replace('<tt><pre>', '')
        df = df[~df['Year'].str.contains('</pre></tt>')]
        df['area'] = file + 1
        df.drop('empty', axis=1, inplace=True)
        frames.append(df)
    
    out_df = pd.concat(frames).drop_duplicates().reset_index(drop=True)
    out_df.drop([22, 12])
    out_df = out_df.loc[(out_df.area != 12) & (out_df.area != 20)]  
    return out_df
 
df = read_and_create_data_frame('/home/kali/Downloads/csv')
print(df)

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   1.0  0.058  254.19  24.78  81.74  53.26     1
1      1982   2.0  0.057  255.70  26.94  78.59  52.76     1
2      1982   3.0  0.056  258.26  27.35  70.35  48.85     1
3      1982   4.0  0.054  260.76  25.28  62.07  43.68     1
4      1982   5.0  0.052  262.59  21.87  59.53  40.70     1
...     ...   ...    ...     ...    ...    ...    ...   ...
59017  2024  48.0  0.100  270.83  57.76  26.51  42.13    27
59018  2024  49.0  0.086  268.23  55.83  29.40  42.61    27
59019  2024  50.0  0.076  266.10  53.85  28.50  41.18    27
59020  2024  51.0  0.068  264.86  53.12  25.29  39.20    27
59021  2024  52.0  0.070  265.65  59.34  19.50  39.42    27

[54650 rows x 8 columns]


3. Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA на потрібні.

In [7]:
def NewIDS(df):
    province_indexes = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
    df.replace({'area':province_indexes}, inplace = True)
    df['area'] = df['area'].replace(state_id_name)
    return df

print(NewIDS(df))

       Year  Week    SMN     SMT    VCI    TCI    VHI      area
0      1982   1.0  0.058  254.19  24.78  81.74  53.26  Cherkasy
1      1982   2.0  0.057  255.70  26.94  78.59  52.76  Cherkasy
2      1982   3.0  0.056  258.26  27.35  70.35  48.85  Cherkasy
3      1982   4.0  0.054  260.76  25.28  62.07  43.68  Cherkasy
4      1982   5.0  0.052  262.59  21.87  59.53  40.70  Cherkasy
...     ...   ...    ...     ...    ...    ...    ...       ...
59017  2024  48.0  0.100  270.83  57.76  26.51  42.13  Zhytomyr
59018  2024  49.0  0.086  268.23  55.83  29.40  42.61  Zhytomyr
59019  2024  50.0  0.076  266.10  53.85  28.50  41.18  Zhytomyr
59020  2024  51.0  0.068  264.86  53.12  25.29  39.20  Zhytomyr
59021  2024  52.0  0.070  265.65  59.34  19.50  39.42  Zhytomyr

[54650 rows x 8 columns]


4. Реалізувати процедури для формування вибірок наступного виду: Ряд VHI для області за рік, пошук екстремумів (min та max);

In [8]:
def find_max_vhi(df, area, year):

  filtered_df = df[ (df['area'] == area) & (df['Year'] == year) ]
  if filtered_df.empty:
    return None  
  return filtered_df['VHI'].max()


def find_min_vhi(df, area, year):

  filtered_df = df[ (df['area'] == area) & (df['Year'] == year) ]
  if filtered_df.empty:
    return None  
  return filtered_df['VHI'].min()

def analyze_vhi(df, year):

  for area in df['area'].unique():
    max_vhi = find_max_vhi(df.copy(), area, year) 
    min_vhi = find_min_vhi(df.copy(), area, year) 
    print(f'{area}, max: {max_vhi}, min:{min_vhi} ')

analyze_vhi(df, '2022')

Cherkasy, max: 56.28, min:37.65 
Chernihiv, max: 81.96, min:40.3 
Chernivtsi, max: 78.8, min:37.84 
Republic of Crimea, max: 62.81, min:36.56 
Dnipropetrovska, max: 53.07, min:32.11 
Donetsk, max: 54.81, min:35.7 
Ivano-Frankivsk, max: 65.14, min:30.32 
Kharkiv, max: 53.83, min:38.47 
Kherson, max: 60.93, min:34.83 
Khmelnytsky, max: 63.71, min:41.93 
Kyiv, max: 56.41, min:37.34 
Kirovohrad, max: 64.19, min:33.37 
Luhansk, max: 58.87, min:38.86 
Lviv, max: 55.41, min:29.62 
Mykolaiv, max: 80.55, min:37.74 
Odesa, max: 73.98, min:36.45 
Poltavska, max: 57.39, min:37.64 
Rivne, max: 61.99, min:38.31 
Sumy, max: 67.19, min:38.47 
Ternopil, max: 61.15, min:38.38 
Zakarpattia, max: 63.81, min:30.36 
Vinnytsia, max: 61.06, min:39.52 
Volyn, max: 70.16, min:39.11 
Zaporizhzhya, max: 72.32, min:36.12 
Zhytomyr, max: 56.9, min:32.28 


5. Ряд VHI за вказаний діапазон років для вказаних областей; виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25); Аналогічно для помірних посух

In [9]:
def drought_years2(df, state_percent, drought_percent_less, drought_percent_more, drought_type):
    drought = df[(df.VHI >= drought_percent_less) & (df.VHI <= drought_percent_more)]
    state_number = (state_percent / 100) * df['area'].nunique()
    yearslist = []

    for year in sorted(df['Year'].unique()):
        drought_per_year = drought[drought['Year'] == year]
        drought_area_number = len(drought_per_year['area'].unique())

        if drought_area_number >= state_number:
            yearslist.append(year)

    if yearslist:
        print(f"Роки, коли {drought_type} посуха була в більш ніж {state_percent}% областей:")
        for year in yearslist:
            print(year)
    else:
        print(f"Роки, коли {drought_type} посуха була в більш ніж {state_percent}% областей не знайдено.")

# Виклик функції для екстремальної посухи (0 <= VHI <= 15)
drought_years2(df, 20, 0, 15, "екстремальна")

# Виклик функції для помірної посухи (15 <= VHI <= 35)
drought_years2(df, 20, 15, 35, "помірна")

Роки, коли екстремальна посуха була в більш ніж 20% областей:
2000
Роки, коли помірна посуха була в більш ніж 20% областей:
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
